# Ocean Temperature Data Exploring

## Setup

Analysis and visualization was done using R and various packages. The following is the script used to generate 2 scatterplot graphs.

In [1]:
library(tidyverse)
library(lubridate) 
library(ggplot2)
library(plotly)
options(repr.plot.width=10, repr.plot.height=6)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang


Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2


-- Attaching packages --------------------------------------- tidyverse 1.2.1 --


v ggplot2 3.1.1       v purrr   0.3.2  
v tibble  2.1.1       v dplyr   0.8.0.1
v tidyr   0.8.3       v stringr 1.4.0  
v readr   1.3.1       v forcats 0.4.0  


-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


...

The following object is masked from 'package:base':

    date




Attaching package: 'plotly'



The following object is masked from 'package:ggplot2':

    last_plot



The following object is masked from 'package:stats':

    filter



The following object is masked from 'package:graphics':

    layout



## Reading and Wrangling Data

Temperature data is found in the `"data"` folder, while coordinates (and the time recorded) are in the `"data/nav"` folder.

All of the temperature data was cleaned using a `Python` script, located in the `data_cleaning` folder. No packages were used, and can be used as long as a v3.9 Python is installed (anything above or below is untested) and the scripts are pointed to the right data sources. *Unfortunately, the version where Jupyter runs on Windows (v3.7 Python) and the data cleaner (v3.9 Python) is different.*

- `data/temp_processed_summarized.csv`: mean temp over min
- `data/temp_processed.csv`: all temp
- `data/nav/nav_processed.csv`: mean GPS over min
- `data/nav_temp_joined_processed.csv`: joined mean temp & GPS over min

The dates are all of type `character`, meaning extracting any use without it being a proper `date` type is hard. Therefore, time and date must be formatted.

In [2]:
format_datetime <- function(df) {
  df_new <- df %>%
    # https://www.neonscience.org/resources/learning-hub/tutorials/dc-time-series-subset-dplyr-r
    mutate(date = as.Date(date, format = '%m/%d/%Y')) %>%
    # https://www.tidyverse.org/blog/2021/03/clock-0-1-0/
    mutate(datetime = as.POSIXct(date, "America/Vancouver")) %>%
    mutate(datetime = datetime +hour(time)+ minute(time))
  
  return(df_new)
}

The following are all of the functions needed to clean up 1 temperature file. However, we have quite a few files, and trying to clean and instantiate each by hand is cumbersome. Therefore, we will iterate through all of the files and summarize.

```{attention}
Please keep in mind that the following code blocks will take pretty long to run.
```

In [3]:
clean_SBE45_data <- function(x) {
  read <- read_delim(x, delim = ",", 
                     col_names = c("date", 
                                    "time", 
                                    "int_temp", 
                                    "conductivity",
                                    "salinity",
                                    "sound_vel",
                                    "ext_temp")) %>%
    select(date, time, ext_temp)
  return(read)
}

In [4]:
clean_STT_TSG_data <- function(x) {
  read <- read_delim(x, delim = ",",
                     col_names = c("date",
                                   "time",
                                   "type",
                                   "diff",
                                   "ext_temp",
                                   "int_temp")) %>%
    select(date, time, ext_temp)    
  return(read)
}

In [5]:
clean_temp_data <- function(x) {
  # https://stackoverflow.com/questions/10128617/test-if-characters-are-in-a-string
  if(grepl("SBE45-TSG-MSG", x, fixed = TRUE)) {
    return(clean_SBE45_data(x))
  } else {
    return(clean_STT_TSG_data(x))
  }
}

This is the data before it is summarized by the minute.

In [6]:
# https://stackoverflow.com/questions/11433432/how-to-import-multiple-csv-files-at-once
all_temperature_loaded <- list.files(path = "data/",
             pattern = "*.Raw",
             full.names = T) %>%
  map_df(~clean_temp_data(.))

Parsed with column specification:
cols(
  date = col_character(),
  time = col_time(format = ""),
  int_temp = col_double(),
  conductivity = col_double(),
  salinity = col_double(),
  sound_vel = col_double(),
  ext_temp = col_double()
)


Parsed with column specification:
cols(
  date = col_character(),
  time = col_time(format = ""),
  int_temp = col_double(),
  conductivity = col_double(),
  salinity = col_double(),
  sound_vel = col_double(),
  ext_temp = col_double()
)


Warning message:
"5 parsing failures.
  row       col expected     actual                                          file
61277 ext_temp  a double t215.2754  'data/SBE45-TSG-MSG_20210614-000001_NEW2.Raw'
61375 sound_vel a double s1506.032  'data/SBE45-TSG-MSG_20210614-000001_NEW2.Raw'
75629 ext_temp  a double t215.4430  'data/SBE45-TSG-MSG_20210614-000001_NEW2.Raw'
76575 ext_temp  a double t215.4469  'data/SBE45-TSG-MSG_20210614-000001_NEW2.Raw'
78521 sound_vel a double sv1507.524 'data/SBE45-TSG-MSG_20210614-000001_NEW2.Raw'
"

Parsed with column specification:
cols(
  date = col_character(),
  time = col_time(format = ""),
  int_temp = col_double(),
  conductivity = col_double(),
  salinity = col_double(),
  sound_vel = col_double(),
  ext_temp = col_double()
)


Warning message:
"437 parsing failures.
 row          col               expected actual                                          file
 762 salinity     no trailing characters        'data/SBE45-TSG-MSG_20210615-000001_NEW2.Raw'
 839 conductivity no trailing characters        'data/SBE45-TSG-MSG_20210615-000001_NEW2.Raw'
 902 salinity     no trailing characters        'data/SBE45-TSG-MSG_20210615-000001_NEW2.Raw'
1120 salinity     no trailing characters        'data/SBE45-TSG-MSG_20210615-000001_NEW2.Raw'
1196 ext_temp     no trailing characters        'data/SBE45-TSG-MSG_20210615-000001_NEW2.Raw'
.... ............ ...................... ...... .............................................
See problems(...) for more details.
"

...

Parsed with column specification:
cols(
  date = col_character(),
  time = col_time(format = ""),
  type = col_character(),
  diff = col_double(),
  ext_temp = col_double(),
  int_temp = col_double()
)


Warning message:
"86282 parsing failures.
row col  expected    actual                                             file
  1  -- 6 columns 7 columns 'data/SST-TSG-Temp-Diff-MSG_20210704-000001.Raw'
  2  -- 6 columns 7 columns 'data/SST-TSG-Temp-Diff-MSG_20210704-000001.Raw'
  3  -- 6 columns 7 columns 'data/SST-TSG-Temp-Diff-MSG_20210704-000001.Raw'
  4  -- 6 columns 7 columns 'data/SST-TSG-Temp-Diff-MSG_20210704-000001.Raw'
  5  -- 6 columns 7 columns 'data/SST-TSG-Temp-Diff-MSG_20210704-000001.Raw'
... ... ......... ......... ................................................
See problems(...) for more details.
"

Parsed with column specification:
cols(
  date = col_character(),
  time = col_time(format = ""),
  type = col_character(),
  diff = col_double(),
  ext_temp = col_double(),
  int_temp = col_double()
)


Warning message:
"86279 parsing failures.
row col  expected    actual                                             file
  1  -- 6 columns 7 columns 'data/SST-TSG-Temp-Diff-MSG_20210705-000001.Raw'
  2  -- 6 columns 7 columns 'data/SST-TSG-Temp-Diff-MSG_20210705-000001.Raw'
  3  -- 6 columns 7 columns 'data/SST-TSG-Temp-Diff-MSG_20210705-000001.Raw'
  4  -- 6 columns 7 columns 'data/SST-TSG-Temp-Diff-MSG_20210705-000001.Raw'
  5  -- 6 columns 7 columns 'data/SST-TSG-Temp-Diff-MSG_20210705-000001.Raw'
... ... ......... ......... ................................................
See problems(...) for more details.
"

Parsed with column specification:
cols(
  date = col_character(),
  time = col_time(format = ""),
  type = col_character(),
  diff = col_double(),
  ext_temp = col_double(),
  int_temp = col_double()
)


In [ ]:
head(all_temperature_loaded)
summary(all_temperature_loaded)

In [ ]:
all_temperature_cleaned <- all_temperature_loaded %>%
  filter(!is.na(ext_temp)) %>%
  filter(ext_temp > 2) %>%
  format_datetime()

write_csv(all_temperature_cleaned, "data/temp_processed.csv")

all_temperature <- group_by(all_temperature_cleaned, datetime) %>%
    summarize(mean_ext = mean(ext_temp, na.rm = TRUE))

write_csv(all_temperature, "data/temp_processed_summarized.csv")

In [ ]:
head(all_temperature)
summary(all_temperature)

All of the navigation files will also be cleaned up and summarized in similar manner to the temperature files.

In [ ]:
clean_nav_data <- function(x) {
  read <- read_csv(x, 
                   col_names = c(
                     "date",
                     "time",
                     "type",
                     "time_num",
                     "lat",
                     "lat_NS",
                     "long",
                     "long_WE",
                     "gps_quality",
                     "num_sat_view",
                     "hort_dil",
                     "ant_alt",
                     "ant_alt_unit",
                     "geoidal",
                     "geoidal_unit",
                     "age_diff",
                     "diff_station",
                     "checksum"
                   )) %>%
    select(date, time, lat, long)
    return(read)
}

In [ ]:
all_nav_loaded <- list.files(path = "data/nav/",
                      pattern = "*.Raw",
                      full.names = T) %>%
  map_df(~clean_nav_data(.)) 

In [ ]:
all_nav <- all_nav_loaded %>%
    format_datetime() %>%
    group_by(datetime) %>%
    summarize(mean_lat = mean(lat), mean_long = mean(long)) %>%
    mutate(mean_lat = mean_lat/100, mean_long = mean_long/100) %>%
    mutate(deg_lat_int = trunc(mean_lat, 0),
           deg_long_int = trunc(mean_long, 0)) %>%
    mutate(deg_lat_dec = round((mean_lat - deg_lat_int) * 10000),
           deg_long_dec = round((mean_long - deg_long_int) * 10000)) %>%
    mutate(mean_deg_lat = deg_lat_int + deg_lat_dec/(60 * 100),
          mean_deg_long = deg_long_int + deg_long_dec / (60 * 100)) %>%
    select(-deg_lat_int, -deg_long_int, -deg_lat_dec, -deg_long_dec)


write_csv(all_nav, "data/nav/nav_processed.csv")

In [ ]:
head(all_nav)
summary(all_nav)

Since we have the date and time (by the minute) of both the temperature and it's coordinates, we can match the two variables together.

In [ ]:
joined_temp_nav <- inner_join(all_temperature, 
                             all_nav,
                             by = c("datetime" = "datetime"))

write_csv(joined_temp_nav, "data/nav_temp_joined_processed.csv")

In [ ]:
head(joined_temp_nav)
summary(joined_temp_nav)

## Visualize the Data

Time and mean temperature plotted on a scatterplot to see temperature changes over time. Notice that since the ship moves in 1 way vs time, the shape of the 2D scatterplot and the 3D plot is very similar.

In [ ]:
time_plot <- ggplot(all_temperature, aes(x = datetime, 
                                         y = mean_ext, 
                                         colour = mean_ext)) +
  geom_point() +
  scale_colour_gradient(low = "blue", high = "red") +
  labs(x = "Date and Time PST", 
       y = "Mean (by the min) Ocean Temperature (celcius)",
       colour = "Mean External Temperature")
time_plot

In [ ]:
p<- plot_ly(joined_temp_nav, 
        x = ~mean_lat, 
        y = ~mean_long,
        z = ~mean_ext,
        color = ~mean_ext) %>%
  add_markers(size = 0.7)

In [ ]:
embed_notebook(p)